In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
stops_with_zip_codes = pd.read_csv('data/bus_stops_with_zip_codes_open_new.csv')
stops = pd.read_csv('data/stops.csv')
census_data = pd.read_csv('data/census_data.csv')
bus_stops = pd.merge(stops_with_zip_codes, stops[['stop_id', 'route_id']], on='stop_id', how='inner')
print(bus_stops['zip_code'].dtype)
print(census_data['zipcode'].dtype)
bus_stops['zip_code'] = bus_stops['zip_code'].astype(str)
bus_stops['stop_id'] = bus_stops['stop_id'].astype(int)
bus_stops.head()

int64
object


,stop_id,latitude,longitude,zip_code,route_id
0,2,42.332324,-71.081252,2119,['01']
1,5,42.336339,-71.077070,2118,['191' 'SL4' 'SL5']
2,7,42.333043,-71.081108,2118,['19' 'CT3']
3,8,42.331202,-71.070724,2118,['10']
4,13,42.329962,-71.057625,2127,['10' '16' '17' '171' '09' '18' 'CT3']


In [36]:
bus_stops.shape

(1072, 5)

In [37]:
result_df = pd.DataFrame()
for idx, row in bus_stops.iterrows():
    # Split the 'route_id', 'stop_id', and 'zip_code' lists
    routes = row['route_id'].split(' ')
    stop_ids = row['stop_id']
    zip_codes = row['zip_code']

    # Create separate rows for each route with corresponding stop IDs and zip codes
    for i, route in enumerate(routes):
        result_df = result_df.append({'Routes': route, 'Stop_ID': stop_ids, 'zip_code': zip_codes}, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [79]:
result_df['zip_code'] = result_df['zip_code'].astype(str).apply(lambda x: '0' + x if len(x) == 4 else x)

In [80]:
result_df.head()

,Routes,Stop_ID,zip_code
0,43,1249.0,02199
1,43,1250.0,02118
2,43,1251.0,02118
3,43,1252.0,02199
4,43,1253.0,02199


In [81]:
grouped_data=result_df.groupby('Routes').agg({'Stop_ID': list, 'zip_code': list}).reset_index()
grouped_data

,Routes,Stop_ID,zip_code
0,1,"[1.0, 2.0, 6.0, 57.0, 58.0, 59.0, 62.0, 63.0, ...","[02119, 02119, 02119, 02118, 02118, 02118, 021..."
1,10,"[1381.0, 1381.0, 1382.0, 1383.0, 1384.0, 1395....","[02118, 02118, 02118, 02118, 02117, 02117, 021..."
2,100,"[5215.0, 5264.0, 5265.0, 5266.0, 5267.0, 5268....","[02145, 02148, 02148, 02148, 02148, 02148, 021..."
3,101,"[2369.0, 2370.0, 2371.0, 2372.0, 2373.0, 2374....","[02474, 02474, 02474, 02474, 02144, 02144, 021..."
4,104,"[5289.0, 5342.0, 5343.0, 5344.0, 5345.0, 5347....","[02148, 02148, 02148, 02148, 02148, 02148, 021..."
...,...,...,...
151,94,"[2369.0, 2370.0, 2371.0, 2372.0, 2373.0, 2374....","[02474, 02474, 02474, 02474, 02144, 02144, 021..."
152,95,"[2362.0, 2363.0, 2364.0, 2365.0, 2366.0, 2367....","[02174, 02174, 02174, 02174, 02474, 02474, 024..."
153,96,"[2076.0, 2300.0, 2304.0, 2305.0, 2307.0, 2310....","[02163, 02140, 02140, 02140, 02138, 02138, 021..."
154,97,"[5047.0, 5048.0, 5049.0, 5050.0, 5051.0, 5054....","[02148, 02148, 02149, 02149, 02149, 02149, 021..."


In [82]:
grouped_data.to_csv('routeTOzipcode_new.csv')

In [83]:
grouped_data.head()

,Routes,Stop_ID,zip_code
0,1,"[1.0, 2.0, 6.0, 57.0, 58.0, 59.0, 62.0, 63.0, ...","[02119, 02119, 02119, 02118, 02118, 02118, 021..."
1,10,"[1381.0, 1381.0, 1382.0, 1383.0, 1384.0, 1395....","[02118, 02118, 02118, 02118, 02117, 02117, 021..."
2,100,"[5215.0, 5264.0, 5265.0, 5266.0, 5267.0, 5268....","[02145, 02148, 02148, 02148, 02148, 02148, 021..."
3,101,"[2369.0, 2370.0, 2371.0, 2372.0, 2373.0, 2374....","[02474, 02474, 02474, 02474, 02144, 02144, 021..."
4,104,"[5289.0, 5342.0, 5343.0, 5344.0, 5345.0, 5347....","[02148, 02148, 02148, 02148, 02148, 02148, 021..."


In [84]:
# Explode the columns with multiple zip codes in both DataFrames
exploded_routes = grouped_data.explode('zip_code')
exploded_neighborhoods = census_data.explode('zipcode')

In [85]:
exploded_routes.head()

,Routes,Stop_ID,zip_code
0,1,"[1.0, 2.0, 6.0, 57.0, 58.0, 59.0, 62.0, 63.0, ...",02119
0,1,"[1.0, 2.0, 6.0, 57.0, 58.0, 59.0, 62.0, 63.0, ...",02119
0,1,"[1.0, 2.0, 6.0, 57.0, 58.0, 59.0, 62.0, 63.0, ...",02119
0,1,"[1.0, 2.0, 6.0, 57.0, 58.0, 59.0, 62.0, 63.0, ...",02118
0,1,"[1.0, 2.0, 6.0, 57.0, 58.0, 59.0, 62.0, 63.0, ...",02118


In [86]:
# Merge exploded DataFrames based on the exploded zip_code column
merged_data = pd.merge(exploded_routes, exploded_neighborhoods, left_on='zip_code', right_on='zipcode')

# Display the resulting merged data
print(merged_data)

     Routes                                            Stop_ID zip_code  \
0         1  [1.0, 2.0, 6.0, 57.0, 58.0, 59.0, 62.0, 63.0, ...    02119   
1         1  [1.0, 2.0, 6.0, 57.0, 58.0, 59.0, 62.0, 63.0, ...    02119   
2         1  [1.0, 2.0, 6.0, 57.0, 58.0, 59.0, 62.0, 63.0, ...    02119   
3         1  [1.0, 2.0, 6.0, 57.0, 58.0, 59.0, 62.0, 63.0, ...    02119   
4         1  [1.0, 2.0, 6.0, 57.0, 58.0, 59.0, 62.0, 63.0, ...    02119   
...     ...                                                ...      ...   
2865     86  [1589.0, 1026.0, 1027.0, 1029.0, 1030.0, 1031....    02134   
2866     86  [1589.0, 1026.0, 1027.0, 1029.0, 1030.0, 1031....    02134   
2867     86  [1589.0, 1026.0, 1027.0, 1029.0, 1030.0, 1031....    02134   
2868     86  [1589.0, 1026.0, 1027.0, 1029.0, 1030.0, 1031....    02134   
2869     86  [1589.0, 1026.0, 1027.0, 1029.0, 1030.0, 1031....    02134   

     tract20_nbhd P0020001 P0020005 P0020006 P0020002 P002aapi P002others  \
0         Roxbury    5

In [87]:
merged_data.columns

Index(['Routes', 'Stop_ID', 'zip_code', 'tract20_nbhd', 'P0020001', 'P0020005',
       'P0020006', 'P0020002', 'P002aapi', 'P002others', 'P0040001',
       'P0040005', 'P0040006', 'P0040002', 'P004aapi', 'P004others',
       'P00x0001', 'P00x0005', 'P00x0006', 'P00x0002', 'P00xaapi',
       'P00xothers', 'hhpop', 'P0050001', 'P0050002', 'P0050003', 'P0050004',
       'P0050005', 'P0050006', 'P0050007', 'P0050008', 'P0050009', 'P0050010',
       'H0010001', 'H0010002', 'H0010003', 'hhsize', 'zipcode'],
      dtype='object')

In [88]:
result = merged_data.groupby('tract20_nbhd').agg({
    'Routes': lambda x: list(set(x))}).reset_index()

In [90]:
result.rename(columns={'tract20_nbhd': 'Neighborhood', 'Routes': 'Routes Serviced'})

,Neighborhood,Routes Serviced
0,Allston,"[501, 51, 66, 64, 57, 86, 70]"
1,Brighton,"[65, 501, 51, 66, 64, 57, 86, 70]"
2,Charlestown,"[8993, 105, 91, 93, 104, 86, 92, 109]"
3,Chinatown,"[39, 43, 15, 501, 504, 505, 7, 9, 741, 57, 749..."
4,East Boston,"[116, 121, 171, 741, 712, 120, 116117, 713, 11..."
5,Fenway,"[65, 47, 19, 747, 9, 28, 22, 57, 708, 55, 8, 60]"
6,Hyde Park,"[39, 50, 3233, 40, 4050, 32, 33, 24]"
7,Jamaica Plain,"[39, 50, 42, 38, 32, 33, 21, 40, 4050, 35, 16,..."
8,Mattapan,"[716, 15, 240, 30, 21, 245, 28, 29, 26, 31, 14..."
9,Mission Hill,"[39, 65, 23, 66, 22, 747, 35, 60, 43, 15, 47, ..."


In [91]:
result.to_csv('NeighbourhoodTORoutesServiced_new.csv')

In [93]:
result2 = merged_data.groupby('Routes').agg({
    'tract20_nbhd': lambda x: list(set(x))}).reset_index()

In [103]:
result2=result2.rename(columns={'tract20_nbhd': 'Neighborhoods', 'Routes': 'Route'})

In [104]:
result2.to_csv('RoutesToNeighbourhoods_new.csv')

In [106]:
result2.shape

(82, 2)

In [107]:
census_data.head()

,tract20_nbhd,P0020001,P0020005,P0020006,P0020002,P002aapi,P002others,P0040001,P0040005,P0040006,...,P0050006,P0050007,P0050008,P0050009,P0050010,H0010001,H0010002,H0010003,hhsize,zipcode
0,field concept,Total:,White alone,Black or African American alone,Hispanic or Latino,"Asian, Native Hawaiian and Pacific Islander al...","Other Races or Multiple Races, all ages",Total:,White alone,Black or African American alone,...,Other institutional facilities,Noninstitutionalized population:,College/University student housing,Military quarters,Other noninstitutional facilities,Total:,Occupied,Vacant,household size,zipcode
1,Allston,24904,12536,1326,3259,6271,1512,23140,11976,1184,...,0,3281,3214,0,67,10748,10027,721,2.156477511,02134
2,Back Bay,18190,13065,690,1208,2410,817,17042,12349,641,...,0,1610,1487,0,123,11524,10006,1518,1.630121927,"02199,02116,"
3,Beacon Hill,9336,7521,252,537,630,396,8603,6980,231,...,0,33,0,0,33,6037,5485,552,1.696080219,"02108,02114,"
4,Brighton,52047,32694,2414,5376,8703,2860,47657,30752,2076,...,56,3796,3493,0,303,23653,22535,1118,2.126292434,02135
